In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
!pip install seaborn

!pip install git+https://github.com/tensorflow/docs

  Cloning https://github.com/tensorflow/docs to /tmp/pip-req-build-lgk9fjad
  Running command git clone -q https://github.com/tensorflow/docs /tmp/pip-req-build-lgk9fjad
  Created wheel for tensorflow-docs: filename=tensorflow_docs-0.0.0-cp36-none-any.whl size=80383 sha256=c4da8f0968f703a4169f2e6c6f8632770983417fa16f58a04f8d86f599e50e88
  Stored in directory: /tmp/pip-ephem-wheel-cache-78roa8xt/wheels/eb/1b/35/fce87697be00d2fc63e0b4b395b0d9c7e391a10e98d9a0d97f
Successfully built tensorflow-docs


In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals

import pathlib

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.preprocessing import LabelEncoder

In [4]:
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers

print(tf.__version__)

TensorFlow 2.x selected.
2.1.0


In [0]:
import tensorflow_docs as tfdocs
import tensorflow_docs.plots
import tensorflow_docs.modeling

In [0]:
dataset_path = "/content/drive/My Drive/data/data.txt"

In [7]:
column_names = ['country', 'year', 'uniqueid', 'bank_account',
                'location_type', 'cellphone_access', 'household_size',
                'age_of_respondent', 'gender_of_respondent', 'relationship_with_head',
                'marital_status', 'education_level', 'job_type']

raw_dataset = pd.read_csv(dataset_path, names=column_names,
                      sep=";", skipinitialspace=True)

dataset = raw_dataset.copy()
dataset.tail()

,country,year,uniqueid,bank_account,location_type,cellphone_access,household_size,age_of_respondent,gender_of_respondent,relationship_with_head,marital_status,education_level,job_type
23520,Uganda,2018,uniqueid_2113,No,Rural,Yes,4,48,Female,Head of Household,Divorced/Seperated,No formal education,Other Income
23521,Uganda,2018,uniqueid_2114,No,Rural,Yes,2,27,Female,Head of Household,Single/Never Married,Secondary education,Other Income
23522,Uganda,2018,uniqueid_2115,No,Rural,Yes,5,27,Female,Parent,Widowed,Primary education,Other Income
23523,Uganda,2018,uniqueid_2116,No,Urban,Yes,7,30,Female,Parent,Divorced/Seperated,Secondary education,Self employed
23524,Uganda,2018,uniqueid_2117,No,Rural,Yes,10,20,Male,Child,Single/Never Married,Secondary education,No Income


In [8]:
dataset.isna().sum()

country                   0
year                      0
uniqueid                  0
bank_account              0
location_type             0
cellphone_access          0
household_size            0
age_of_respondent         0
gender_of_respondent      0
relationship_with_head    0
marital_status            0
education_level           0
job_type                  0
dtype: int64

In [9]:
dataset.pop("uniqueid")

0             uniqueid
1           uniqueid_1
2           uniqueid_2
3           uniqueid_3
4           uniqueid_4
             ...      
23520    uniqueid_2113
23521    uniqueid_2114
23522    uniqueid_2115
23523    uniqueid_2116
23524    uniqueid_2117
Name: uniqueid, Length: 23525, dtype: object

In [0]:
dataset['age_of_respondent'] = pd.to_numeric(dataset['age_of_respondent'], errors='coerce')
dataset['household_size'] = pd.to_numeric(dataset['household_size'], errors='coerce')


In [11]:
dataset = pd.get_dummies(dataset, prefix='', prefix_sep='', columns=["country", "year", "relationship_with_head", "marital_status", "education_level", "job_type"])
labelencoder = LabelEncoder()
dataset.iloc[:, 0] = labelencoder.fit_transform(dataset.iloc[:, 0].values)
dataset.iloc[:, 1] = labelencoder.fit_transform(dataset.iloc[:, 1].values)
dataset.iloc[:, 2] = labelencoder.fit_transform(dataset.iloc[:, 2].values)
dataset.iloc[:, 5] = labelencoder.fit_transform(dataset.iloc[:, 5].values)
dataset.tail()

,bank_account,location_type,cellphone_access,household_size,age_of_respondent,gender_of_respondent,Kenya,Rwanda,Tanzania,Uganda,country,2016,2017,2018,year,Child,Head of Household,Other non-relatives,Other relative,Parent,Spouse,relationship_with_head,Divorced/Seperated,Dont know,Married/Living together,Single/Never Married,Widowed,marital_status,No formal education,Other/Dont know/RTA,Primary education,Secondary education,Tertiary education,Vocational/Specialised training,education_level,Dont Know/Refuse to answer,Farming and Fishing,Formally employed Government,Formally employed Private,Government Dependent,Informally employed,No Income,Other Income,Remittance Dependent,Self employed,job_type
23520,0,0,1,4.0,48.0,0,0,0,0,1,0,0,0,1,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
23521,0,0,1,2.0,27.0,0,0,0,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0
23522,0,0,1,5.0,27.0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
23523,0,1,1,7.0,30.0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0
23524,0,0,1,10.0,20.0,1,0,0,0,1,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0


In [12]:
dataset.pop("country")
dataset.pop("year")
dataset.pop("relationship_with_head")
dataset.pop("marital_status")
dataset.pop("education_level")
dataset.pop("job_type")
dataset.tail()

,bank_account,location_type,cellphone_access,household_size,age_of_respondent,gender_of_respondent,Kenya,Rwanda,Tanzania,Uganda,2016,2017,2018,Child,Head of Household,Other non-relatives,Other relative,Parent,Spouse,Divorced/Seperated,Dont know,Married/Living together,Single/Never Married,Widowed,No formal education,Other/Dont know/RTA,Primary education,Secondary education,Tertiary education,Vocational/Specialised training,Dont Know/Refuse to answer,Farming and Fishing,Formally employed Government,Formally employed Private,Government Dependent,Informally employed,No Income,Other Income,Remittance Dependent,Self employed
23520,0,0,1,4.0,48.0,0,0,0,0,1,0,0,1,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
23521,0,0,1,2.0,27.0,0,0,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0
23522,0,0,1,5.0,27.0,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0
23523,0,1,1,7.0,30.0,0,0,0,0,1,0,0,1,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1
23524,0,0,1,10.0,20.0,1,0,0,0,1,0,0,1,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0


In [0]:
train_dataset = dataset.sample(frac=1,random_state=0)
test_dataset = dataset.drop(train_dataset.index)

In [14]:
train_stats = train_dataset.describe()
train_stats.pop("bank_account")
train_stats = train_stats.transpose()
train_stats

,count,mean,std,min,25%,50%,75%,max
location_type,23525.0,0.390351,0.487926,0.0,0.0,0.0,1.0,2.0
cellphone_access,23525.0,0.742019,0.437630,0.0,0.0,1.0,1.0,2.0
household_size,23524.0,3.797483,2.227613,1.0,2.0,3.0,5.0,21.0
age_of_respondent,23524.0,38.805220,16.520569,16.0,26.0,35.0,49.0,100.0
gender_of_respondent,23525.0,0.410159,0.491959,0.0,0.0,0.0,1.0,2.0
Kenya,23525.0,0.257938,0.437509,0.0,0.0,0.0,1.0,1.0
Rwanda,23525.0,0.371307,0.483165,0.0,0.0,0.0,1.0,1.0
Tanzania,23525.0,0.281403,0.449693,0.0,0.0,0.0,1.0,1.0
Uganda,23525.0,0.089309,0.285196,0.0,0.0,0.0,0.0,1.0
2016,23525.0,0.371307,0.483165,0.0,0.0,0.0,1.0,1.0


In [15]:
train_labels = train_dataset.pop('bank_account')
train_labels

4857     0
22132    0
22676    0
5165     0
16211    0
        ..
13123    0
19648    0
9845     1
10799    0
2732     1
Name: bank_account, Length: 23525, dtype: int64

In [16]:
def norm(x):
  return (x - train_stats['mean']) / train_stats['std']
normed_train_data = norm(train_dataset)
normed_train_data

,location_type,cellphone_access,household_size,age_of_respondent,gender_of_respondent,Kenya,Rwanda,Tanzania,Uganda,2016,2017,2018,Child,Head of Household,Other non-relatives,Other relative,Parent,Spouse,Divorced/Seperated,Dont know,Married/Living together,Single/Never Married,Widowed,No formal education,Other/Dont know/RTA,Primary education,Secondary education,Tertiary education,Vocational/Specialised training,Dont Know/Refuse to answer,Farming and Fishing,Formally employed Government,Formally employed Private,Government Dependent,Informally employed,No Income,Other Income,Remittance Dependent,Self employed
4857,1.24947,0.589496,-1.255821,-0.290863,1.198962,1.696105,-0.768490,-0.625766,-0.313151,-0.768490,-0.625766,1.371025,-0.323517,0.912916,-0.090233,-0.17095,-0.219991,-0.619193,3.214257,-0.018444,-0.917228,-0.716672,-0.360667,-0.487336,-0.0386,0.916050,-0.467735,-0.227428,-0.187986,-0.073380,-0.548508,-0.129325,-0.216678,-0.103007,1.789695,-0.165472,-0.219353,-0.346900,-0.613744
22132,-0.80002,0.589496,0.539823,-0.532985,1.198962,-0.589561,-0.768490,-0.625766,3.193214,-0.768490,-0.625766,1.371025,-0.323517,0.912916,-0.090233,-0.17095,-0.219991,-0.619193,-0.311101,-0.018444,1.090195,-0.716672,-0.360667,-0.487336,-0.0386,0.916050,-0.467735,-0.227428,-0.187986,13.627122,-0.548508,-0.129325,-0.216678,-0.103007,-0.558731,-0.165472,-0.219353,-0.346900,-0.613744
22676,-0.80002,-1.695540,0.988734,0.132851,-0.833726,-0.589561,-0.768490,-0.625766,3.193214,-0.768490,-0.625766,1.371025,-0.323517,-1.095345,-0.090233,-0.17095,-0.219991,1.614936,-0.311101,-0.018444,1.090195,-0.716672,-0.360667,2.051885,-0.0386,-1.091597,-0.467735,-0.227428,-0.187986,-0.073380,-0.548508,-0.129325,-0.216678,-0.103007,-0.558731,-0.165472,-0.219353,-0.346900,1.629276
5165,1.24947,0.589496,0.090912,0.072321,-0.833726,1.696105,-0.768490,-0.625766,-0.313151,-0.768490,-0.625766,1.371025,-0.323517,-1.095345,-0.090233,-0.17095,-0.219991,1.614936,-0.311101,-0.018444,1.090195,-0.716672,-0.360667,-0.487336,-0.0386,-1.091597,2.137870,-0.227428,-0.187986,-0.073380,-0.548508,7.732113,-0.216678,-0.103007,-0.558731,-0.165472,-0.219353,-0.346900,-0.613744
16211,-0.80002,0.589496,-0.806910,-0.775108,1.198962,-0.589561,-0.768490,1.597973,-0.313151,-0.768490,1.597973,-0.729350,-0.323517,0.912916,-0.090233,-0.17095,-0.219991,-0.619193,-0.311101,-0.018444,-0.917228,1.395279,-0.360667,-0.487336,-0.0386,-1.091597,-0.467735,4.396810,-0.187986,-0.073380,-0.548508,-0.129325,4.614940,-0.103007,-0.558731,-0.165472,-0.219353,-0.346900,-0.613744
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13123,-0.80002,0.589496,-0.806910,2.130361,-0.833726,-0.589561,1.301198,-0.625766,-0.313151,1.301198,-0.625766,-0.729350,-0.323517,0.912916,-0.090233,-0.17095,-0.219991,-0.619193,-0.311101,-0.018444,-0.917228,1.395279,-0.360667,2.051885,-0.0386,-1.091597,-0.467735,-0.227428,-0.187986,-0.073380,-0.548508,-0.129325,-0.216678,-0.103007,-0.558731,-0.165472,-0.219353,2.882552,-0.613744
19648,1.24947,-1.695540,0.090912,-0.411924,-0.833726,-0.589561,-0.768490,1.597973,-0.313151,-0.768490,1.597973,-0.729350,-0.323517,-1.095345,-0.090233,-0.17095,-0.219991,1.614936,-0.311101,-0.018444,-0.917228,1.395279,-0.360667,-0.487336,-0.0386,0.916050,-0.467735,-0.227428,-0.187986,-0.073380,-0.548508,-0.129325,-0.216678,-0.103007,-0.558731,6.043044,-0.219353,-0.346900,-0.613744
9845,1.24947,0.589496,-0.357999,-0.714577,1.198962,-0.589561,1.301198,-0.625766,-0.313151,1.301198,-0.625766,-0.729350,3.090899,-1.095345,-0.090233,-0.17095,-0.219991,-0.619193,-0.311101,-0.018444,-0.917228,1.395279,-0.360667,-0.487336,-0.0386,-1.091597,2.137870,-0.227428,-0.187986,-0.073380,-0.548508,-0.129325,-0.216678,-0.103007,1.789695,-0.165472,-0.219353,-0.346900,-0.613744
10799,-0.80002,-1.695540,0.988734,0.374974,-0.833726,-0.589561,1.301198,-0.625766,-0.313151,1.301198,-0.625766,-0.729350,-0.323517,-1.095345,-0.090233,-0.17095,

In [0]:
def build_model():
  model = keras.Sequential([
    layers.Dense(64, activation='relu', input_shape=[len(train_dataset.keys())]),
    layers.Dense(64, activation='relu'),
    layers.Dense(1, activation='sigmoid')
  ])

  optimizer = tf.keras.optimizers.RMSprop(0.001)

  model.compile(optimizer='adam',
                loss='binary_crossentropy',
                metrics=['accuracy'])
  return model

In [0]:
model = build_model()

In [19]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 64)                2560      
_________________________________________________________________
dense_1 (Dense)              (None, 64)                4160      
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 65        
Total params: 6,785
Trainable params: 6,785
Non-trainable params: 0
_________________________________________________________________


In [20]:

example_batch = normed_train_data[:10]

example_result = model.predict(example_batch)
example_result

array([[0.3500079 ],
       [0.22684759],
       [0.31130576],
       [0.65313107],
       [0.38192418],
       [0.46963295],
       [0.27980524],
       [0.41709974],
       [0.38893056],
       [0.44823536]], dtype=float32)

In [0]:
EPOCHS = 300

history = model.fit(
  normed_train_data, train_labels,
  epochs=EPOCHS, validation_split = 0.2, verbose=0,
  callbacks=[tfdocs.modeling.EpochDots()])


Epoch: 0, accuracy:0.8649,  loss:0.3966,  val_accuracy:0.8597,  val_loss:0.4055,  
....................................................................................................
Epoch: 100, accuracy:0.8590,  loss:0.4071,  val_accuracy:0.8597,  val_loss:0.4055,  
....................................................................................................
Epoch: 200, accuracy:0.8590,  loss:0.4068,  val_accuracy:0.8597,  val_loss:0.4061,  
...........

In [0]:
loss, mae = model.evaluate(normed_train_data, train_labels, verbose=2)

print("Testing set Mean Abs Error: {:5.2f} MPG".format(mae))

23525/23525 - 1s - loss: nan - accuracy: 0.8592
Testing set Mean Abs Error:  0.86 MPG


In [0]:
dataset_path = "/content/drive/My Drive/data/test.txt"

In [0]:
column_names = ['country', 'year', 'uniqueid',
                'location_type', 'cellphone_access', 'household_size',
                'age_of_respondent', 'gender_of_respondent', 'relationship_with_head',
                'marital_status', 'education_level', 'job_type']

raw_dataset = pd.read_csv(dataset_path, names=column_names,
                      sep=";", skipinitialspace=True)

dataset = raw_dataset.copy()
dataset.tail()

,country,year,uniqueid,location_type,cellphone_access,household_size,age_of_respondent,gender_of_respondent,relationship_with_head,marital_status,education_level,job_type
10082,Uganda,2018,uniqueid_2998,Rural,No,2,62,Female,Spouse,Married/Living together,Primary education,Self employed
10083,Uganda,2018,uniqueid_2999,Urban,Yes,8,42,Male,Head of Household,Married/Living together,Primary education,Self employed
10084,Uganda,2018,uniqueid_3000,Urban,Yes,1,39,Male,Head of Household,Single/Never Married,Secondary education,Other Income
10085,Uganda,2018,uniqueid_3001,Rural,Yes,6,28,Female,Spouse,Married/Living together,Primary education,Self employed
10086,Uganda,2018,uniqueid_3002,Urban,Yes,7,16,Male,Other relative,Single/Never Married,Secondary education,Other Income


In [0]:
dataset = pd.get_dummies(dataset, prefix='', prefix_sep='', columns=["year", "relationship_with_head", "marital_status", "education_level", "job_type"])
labelencoder = LabelEncoder()
dataset.iloc[:, 0] = labelencoder.fit_transform(dataset.iloc[:, 0].values)
dataset.iloc[:, 1] = labelencoder.fit_transform(dataset.iloc[:, 1].values)
dataset.iloc[:, 2] = labelencoder.fit_transform(dataset.iloc[:, 2].values)
dataset.iloc[:, 5] = labelencoder.fit_transform(dataset.iloc[:, 5].values)
dataset.tail()

,country,location_type,cellphone_access,household_size,age_of_respondent,gender_of_respondent,2016,2017,2018,year,Child,Head of Household,Other non-relatives,Other relative,Parent,Spouse,relationship_with_head,Divorced/Seperated,Dont know,Married/Living together,Single/Never Married,Widowed,marital_status,No formal education,Other/Dont know/RTA,Primary education,Secondary education,Tertiary education,Vocational/Specialised training,education_level,Dont Know/Refuse to answer,Farming and Fishing,Formally employed Government,Formally employed Private,Government Dependent,Informally employed,No Income,Other Income,Remittance Dependent,Self employed,job_type
10082,3,0,0,2.0,62.0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
10083,3,1,1,8.0,42.0,1,0,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
10084,3,1,1,1.0,39.0,1,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0
10085,3,0,1,6.0,28.0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
10086,3,1,1,7.0,16.0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0


In [0]:
dataset.pop("year")
dataset.pop("relationship_with_head")
dataset.pop("marital_status")
dataset.pop("education_level")
dataset.pop("job_type")
dataset.tail()

,country,location_type,cellphone_access,household_size,age_of_respondent,gender_of_respondent,2016,2017,2018,Child,Head of Household,Other non-relatives,Other relative,Parent,Spouse,Divorced/Seperated,Dont know,Married/Living together,Single/Never Married,Widowed,No formal education,Other/Dont know/RTA,Primary education,Secondary education,Tertiary education,Vocational/Specialised training,Dont Know/Refuse to answer,Farming and Fishing,Formally employed Government,Formally employed Private,Government Dependent,Informally employed,No Income,Other Income,Remittance Dependent,Self employed
10082,3,0,0,2.0,62.0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1
10083,3,1,1,8.0,42.0,1,0,0,1,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1
10084,3,1,1,1.0,39.0,1,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0
10085,3,0,1,6.0,28.0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1
10086,3,1,1,7.0,16.0,1,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0


In [0]:
train_stats = train_dataset.describe()
train_stats = train_stats.transpose()
train_stats

,count,mean,std,min,25%,50%,75%,max
location_type,10087.0,0.386537,0.487184,0.0,0.0,0.0,1.0,2.0
cellphone_access,10087.0,0.749579,0.433506,0.0,0.0,1.0,1.0,2.0
household_size,10087.0,2.779816,2.214458,0.0,1.0,2.0,4.0,18.0
age_of_respondent,10086.0,38.308348,16.270053,16.0,26.0,35.0,48.0,100.0
gender_of_respondent,10087.0,0.257857,0.437476,0.0,0.0,0.0,1.0,1.0
Kenya,10087.0,0.257857,0.437476,0.0,0.0,0.0,1.0,1.0
Rwanda,10087.0,0.371270,0.483168,0.0,0.0,0.0,1.0,1.0
Tanzania,10087.0,0.281451,0.449729,0.0,0.0,0.0,1.0,1.0
Uganda,10087.0,0.089323,0.285223,0.0,0.0,0.0,0.0,1.0
2016,10087.0,0.371270,0.483168,0.0,0.0,0.0,1.0,1.0
